Title : **Product attribute Extraction**

Group Members:-

Manas pandey :- 203100065

Mudit sand :- 203100068

Supriyo Roy :- 193109013

Library

In [ ]:
import tensorflow as tf
#from tensorflow.keras.layers import Input, Dense, Embedding, Flatten 
from tensorflow.keras.models import Model
#from tensorflow.keras.optimizers import SGD
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.initializers import Constant


Dataset

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Dataset - Sheet1.csv')


Dataset Visualization

In [ ]:
dataset.head()

,Unnamed: 0,title,brand,present,root_cat
0,0,Galt Shape Puzzle Emergency Vehicles Baby/Todd...,Galt,True,2984
1,1,BRIO Magnetic Building Wooden Blocks Toddler A...,Brio,True,2984
2,2,NEW GRAFIX 6PCS METALLIC FABRIC PAINT PEN MARK...,GRAFIX,True,550
3,3,NEW GRAFIX 6PCS METALLIC FABRIC PAINT PEN MARK...,GRAFIX,True,550
4,4,Coca Cola Coke Beach Girl Large 24 x 36 Wall P...,Coca Cola,True,550


Taking the important features of the dataset for further processing

In [ ]:
dataset= dataset[['title','brand']]
dataset.head()

,title,brand
0,Galt Shape Puzzle Emergency Vehicles Baby/Todd...,Galt
1,BRIO Magnetic Building Wooden Blocks Toddler A...,Brio
2,NEW GRAFIX 6PCS METALLIC FABRIC PAINT PEN MARK...,GRAFIX
3,NEW GRAFIX 6PCS METALLIC FABRIC PAINT PEN MARK...,GRAFIX
4,Coca Cola Coke Beach Girl Large 24 x 36 Wall P...,Coca Cola


Tokenizer

In [ ]:
featurestoCare = 20000
MAX_SEQUENCE_LENGTH=50
tokens = Tokenizer(num_words=featurestoCare)
tokens.fit_on_texts(dataset['title'].values)
x = tokens.texts_to_sequences(dataset['title'].values)
x = pad_sequences(x)
tokens.fit_on_texts(dataset['brand'].values)
y = tokens.texts_to_sequences(dataset['brand'].values)
y = pad_sequences(y)
y=y[:,2:]

In [ ]:
x.shape

(1509, 21)

Train Test Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)


Word Embedding

In [ ]:
embeddings_index = {}
f = open('/content/drive/MyDrive/myfile.txt') #Embedding File

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
word_index = tokens.word_index
print('Found %s unique tokens.' % len(word_index))

Found 3666 unique tokens.


In [ ]:
# Word Embedding 
embedding_dim = 1000
num_words = min(featurestoCare, len(word_index)) + 1
# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))
# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector

In [ ]:
# Embedding Layer
embedding_layer=Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    trainable=True)

CNN Model

In [ ]:
# CNN Model 
sequence_input = Input(shape=(21,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

#Convolution Layer
x = Conv1D(128, 3, activation='relu')(embedded_sequences)

x = Conv1D(64, 3, activation='relu')(x)

x = Conv1D(64, 3, activation='relu')(x)
x = Conv1D(64, 3, activation='relu')(x)

x = Flatten()(x)

#Dense Layer
x = Dense(64, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)

#Optimizer
sgd=SGD(learning_rate=1e-5, momentum=0.5)

#Model Compilation
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['acc'])

#Model Fitting
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=10, batch_size=32)

Epoch 1/10
43/43 [==============================] - 4s 74ms/step - loss: 181.1569 - acc: 0.8686 - val_loss: 209.7675 - val_acc: 0.9801
Epoch 2/10
43/43 [==============================] - 3s 69ms/step - loss: 184.6107 - acc: 0.9692 - val_loss: 199.9536 - val_acc: 0.9801
Epoch 3/10
43/43 [==============================] - 3s 68ms/step - loss: 160.1324 - acc: 0.9591 - val_loss: 191.8774 - val_acc: 0.9801
Epoch 4/10
43/43 [==============================] - 3s 68ms/step - loss: 159.9297 - acc: 0.9672 - val_loss: 185.4232 - val_acc: 0.9801
Epoch 5/10
43/43 [==============================] - 3s 69ms/step - loss: 144.2470 - acc: 0.9594 - val_loss: 180.6547 - val_acc: 0.9801
Epoch 6/10
43/43 [==============================] - 3s 69ms/step - loss: 137.2668 - acc: 0.9649 - val_loss: 177.5742 - val_acc: 0.9801
Epoch 7/10
43/43 [==============================] - 3s 70ms/step - loss: 142.9906 - acc: 0.9623 - val_loss: 175.9934 - val_acc: 0.9801
Epoch 8/10
43/43 [==============================] - 3s 

Model Summary

In [ ]:
print(model.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 21, 1000)          3667000   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 19, 128)           384128    
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 17, 64)            24640     
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 15, 64)            12352     
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 13, 64)            12352     
_________________________________________________________________
flatten_4 (Flatten)          (None, 832)               0   

Prediction

In [ ]:
pred=model.predict(X_test)


In [ ]:
print(pred)

[[0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075 0.8187925]
 [0.1812075